**Install pycaret into environment**

In [ ]:
!pip install uv
!uv pip install pycaret[full] --system

Streaming output truncated to the last 5000 lines.
flask-simplelogin ------------------------------ 7.26 kB/7.26 kB
wurlitzer  ------------------------------ 8.59 kB/8.59 kB
xmltodict  ------------------------------ 9.97 kB/9.97 kB
iterative-telemetry ------------------------------ 10.62 kB/10.62 kB
multimethod ------------------------------ 10.65 kB/10.65 kB
deprecation ------------------------------ 11.18 kB/11.18 kB
colorlog   ------------------------------ 11.36 kB/11.36 kB
percy      ------------------------------ 12.00 kB/12.00 kB
flask-wtf  ------------------------------ 12.72 kB/12.72 kB
dacite     ------------------------------ 14.31 kB/14.31 kB
dash-auth  ------------------------------ 14.43 kB/14.43 kB
m2cgen     ------------------------------ 81.92 kB/92.24 kB
visions    ------------------------------ 65.54 kB/104.83 kB
ydata-profiling ------------------------------ 356.19 kB/356.19 kB
litestar   ------------------------------ 357.45 kB/538.37 kB
jedi       ----------------

**Mount google drive and copy kaggle.json to machine**

In [ ]:
from google.colab import drive
drive.mount('mount')

Mounted at mount


In [ ]:
!cp mount/MyDrive/kaggle.json .
!ls

kaggle.json  mount  sample_data


**Acquire the covid-19 cases dataset from kaggle and extract zip**


I am using this dataset last updated 3 days ago on kaggle:

https://www.kaggle.com/datasets/fireballbyedimyrnmom/us-counties-covid-19-dataset

In [ ]:
!KAGGLE_CONFIG_DIR=$(pwd) kaggle datasets download fireballbyedimyrnmom/us-counties-covid-19-dataset
!unzip -o us-counties-covid-19-dataset.zip

Dataset URL: https://www.kaggle.com/datasets/fireballbyedimyrnmom/us-counties-covid-19-dataset
License(s): other
 91% 25.0M/27.5M [00:02<00:00, 15.4MB/s]
100% 27.5M/27.5M [00:02<00:00, 10.4MB/s]
Archive:  us-counties-covid-19-dataset.zip
  inflating: us-counties.csv         


**Load the csv into a pandas dataframe**

In [ ]:
import pandas as pd
data = pd.read_csv('us-counties.csv')
data

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
2502827,2022-05-13,Sweetwater,Wyoming,56037.0,11088,126.0
2502828,2022-05-13,Teton,Wyoming,56039.0,10074,16.0
2502829,2022-05-13,Uinta,Wyoming,56041.0,5643,39.0
2502830,2022-05-13,Washakie,Wyoming,56043.0,2358,44.0


In [ ]:
data['state'] = data['state'].astype('string')
data = data.loc[data['state'] == 'California']
data.drop('state', axis=1, inplace=True)
data.drop('county', axis=1, inplace=True)
data.drop('fips', axis=1, inplace=True)

data['date'] = pd.to_datetime(data['date'], format="%Y-%m-%d")
data

<ipython-input-6-9c3f4f8b43f2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('state', axis=1, inplace=True)
<ipython-input-6-9c3f4f8b43f2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('county', axis=1, inplace=True)
<ipython-input-6-9c3f4f8b43f2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop('fips', axis=1, inplace=True)
<ipython-input-6-9c3f4f8b43f2>:7: SettingWithCopyWarning: 
A value is trying

,date,cases,deaths
5,2020-01-25,1,0.0
9,2020-01-26,1,0.0
10,2020-01-26,1,0.0
14,2020-01-27,1,0.0
15,2020-01-27,1,0.0
...,...,...,...
2499814,2022-05-13,136125,1469.0
2499815,2022-05-13,13758,182.0
2499816,2022-05-13,186062,1498.0
2499817,2022-05-13,41061,316.0


In [ ]:
data = data.groupby(data.date, as_index=False).sum()
data

,date,cases,deaths
0,2020-01-25,1,0.0
1,2020-01-26,2,0.0
2,2020-01-27,2,0.0
3,2020-01-28,2,0.0
4,2020-01-29,2,0.0
...,...,...,...
835,2022-05-09,9297615,90712.0
836,2022-05-10,9322169,90822.0
837,2022-05-11,9325522,90804.0
838,2022-05-12,9329044,90818.0


In [ ]:
data['cases'] = data['cases'].diff()
data['deaths'] = data['deaths'].diff()

data.fillna(0, inplace=True)
data

,date,cases,deaths
0,2020-01-25,0.0,0.0
1,2020-01-26,1.0,0.0
2,2020-01-27,0.0,0.0
3,2020-01-28,0.0,0.0
4,2020-01-29,0.0,0.0
...,...,...,...
835,2022-05-09,7060.0,6.0
836,2022-05-10,24554.0,110.0
837,2022-05-11,3353.0,-18.0
838,2022-05-12,3522.0,14.0


In [ ]:
data.index = pd.PeriodIndex(data.date, freq='D')
data.drop('date', axis=1, inplace=True)
data

,cases,deaths
date,,
2020-01-25,0.0,0.0
2020-01-26,1.0,0.0
2020-01-27,0.0,0.0
2020-01-28,0.0,0.0
2020-01-29,0.0,0.0
...,...,...
2022-05-09,7060.0,6.0
2022-05-10,24554.0,110.0
2022-05-11,3353.0,-18.0


**Import pycaret and setup a univariate forecasting experiment without exogenous variables**




In [ ]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()

# init setup on exp
exp.setup(data['deaths'], target='deaths', fh = 30, session_id = 1337, use_gpu=True)

,Description,Value
0,session_id,1337
1,Target,deaths
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(840, 1)"
5,Transformed data shape,"(840, 1)"
6,Transformed train set shape,"(810, 1)"
7,Transformed test set shape,"(30, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Th

**Train models**

In [ ]:
best = exp.compare_models(errors="raise")

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
stlf,STLF,1.4881,0.9713,61.0483,84.3502,8441460467789152.0000,0.5611,-0.7338,0.0700
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.5941,0.9526,65.4573,82.7216,11977864421460128.0000,0.6067,-0.5227,0.8033
ets,ETS,1.6691,0.9370,68.2808,81.1602,19161487594738428.0000,0.6080,-0.1403,0.1033
exp_smooth,Exponential Smoothing,1.7099,0.9717,69.9674,84.1999,18963729780612428.0000,0.6054,-0.2399,0.2100
auto_arima,Auto ARIMA,1.7193,0.9788,70.3903,84.8965,20689257833291844.0000,0.6258,-0.2898,141.4367
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,1.7252,1.0200,70.5782,88.5117,14507111396785922.0000,0.6427,-0.3644,1.1533
snaive,Seasonal Naive Forecaster,1.7295,1.0642,70.9000,92.4318,9907919180215090.0000,0.7255,-0.9303,0.0867
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.7306,0.9866,70.9774,85.6549,15339454669223936.0000,0.6482,-0.5013,0.8367
arima,ARIMA,1.7415,1.0693,71.4194,92.8705,10770379411230266.0000,0.6599,-0.9892,0.1433
ridge_cds_dt,Ridge w/ Cond. Deseasonalize & Detrending,1.8391,1.0205,75.4940,88.6115,18161256611530212.0000,0.6665,-0.7408,0.7967


Processing:   0%|          | 0/121 [00:00<?, ?it/s]

**Plot Model Predictions**

In [ ]:
exp.plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 30})

**Import pycaret and setup a univariate forecasting experiment with exogenous variables**

In [ ]:
from pycaret.time_series import TSForecastingExperiment
exp = TSForecastingExperiment()

# init setup on exp
exp.setup(data, target='deaths', fh = 30, session_id = 1337, use_gpu=True)

,Description,Value
0,session_id,1337
1,Target,deaths
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(840, 2)"
5,Transformed data shape,"(840, 2)"
6,Transformed train set shape,"(810, 2)"
7,Transformed test set shape,"(30, 2)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] Th

In [ ]:
best = exp.compare_models(errors="raise")

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,1.6409,1.0632,66.9550,92.2135,5204300004206820.0000,0.6600,-0.2928,1.2833
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,1.9952,1.0893,81.5748,94.4291,23954080111120156.0000,0.6847,-0.4659,1.3533
catboost_cds_dt,CatBoost Regressor w/ Cond. Deseasonalize & Detrending,2.0913,1.1872,85.1804,103.0790,9000696240009196.0000,0.6833,-0.8023,11.0000
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,2.1961,1.1599,89.8840,100.7339,17549524084921402.0000,0.7204,-1.0074,1.2900
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,2.1967,1.3446,89.7126,116.7858,10595647910336980.0000,0.6931,-1.4285,1.0333
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,2.2111,1.2055,90.2386,104.5647,18835762149631300.0000,0.7055,-0.7374,1.7867
croston,Croston,2.2240,1.2404,91.1084,107.7111,25895126798200196.0000,0.7475,-1.3189,0.0367
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,2.2314,1.2185,91.4137,105.8125,17493982507939944.0000,0.7198,-1.3838,2.4300
arima,ARIMA,2.2348,1.3449,91.2836,116.8838,10058278196591728.0000,0.8564,-1.7567,0.2200
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.3458,1.2630,95.4149,109.4721,19007573214681252.0000,0.6832,-0.7174,1.1867


Processing:   0%|          | 0/89 [00:00<?, ?it/s]

In [ ]:
exp.plot_model(best, plot = 'forecast', data_kwargs = {'fh' : 30})